In [ ]:
# import libraries

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig 
import scipy.linalg as la


In [ ]:
Aeig = np.array([.9, .92, -.8])
C = np.array([[1,1,1],[2,1,-1]])

# T is the matrix of eigenvectors of A
T = np.array([[1,1,2],[0,1,0],[1,1,1]])
A = T @ np.diag(Aeig) @ la.inv(T)

# R is the covariance matrix of the measurement noise
R = .1*np.array([[1,1],[1,2]])
# Q is the covariance matrix of the process noise
Q = .1*np.eye(3)

# initial state
ny = 2
nx = 3
N = 100
x = np.zeros((nx,N))
v = np.zeros((ny,N))
w = np.zeros((nx,N))
y = np.zeros((ny,N))

for k in range(N-1):
    w[:,k] = np.random.multivariate_normal(np.zeros(nx),Q)
    x[:,k+1] = A @ x[:,k] + w[:,k]
    v[:,k] = np.random.multivariate_normal(np.zeros(ny),R)
    y[:,k] = C @ x[:,k] + v[:,k]

# set up variables for Kalman filter
xhat = np.zeros((nx,N))
xPred = np.zeros((nx,N))
P = np.zeros((nx,nx,N))
PPred = np.zeros((nx,nx,N))
K = np.zeros((nx,ny,N))

xhat[:,0] = np.array([1,1,1])
T = np.random.randn(nx,nx)
P[:,:,0] = T @ T.T



In [ ]:
# Kalman filter process

for k in np.arange(1,N):
    # prediction step
    xPred[:,k] = A @ xhat[:,k-1]
    PPred[:,:,k] = A @ P[:,:,k-1] @ A.T + Q
    
    # update step
    K[:,:,k] = PPred[:,:,k] @ C.T @ la.inv(C @ PPred[:,:,k] @ C.T + R)
    xhat[:,k] = xPred[:,k] + K[:,:,k] @ (y[:,k] - C @ xPred[:,k])
    P[:,:,k] = (np.eye(nx) - K[:,:,k] @ C) @ PPred[:,:,k]

In [ ]:
time = np.arange(N)
plt.plot(time,x[0,:],'b',label='true')
plt.plot(time,xhat[0,:],'r',label='estimate')

plt.figure()
plt.plot(time,x[1,:],'b',label='true')
plt.plot(time,xhat[1,:],'r',label='estimate')

plt.figure()
plt.plot(time,x[2,:],'b',label='true')
plt.plot(time,xhat[2,:],'r',label='estimate')